In [3]:
%load_ext lab_black

In [4]:
import os
import pytz
import pathlib
import requests
from datetime import datetime
from bs4 import BeautifulSoup

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [5]:
r = requests.get("http://sierracounty.ca.gov/628/Sierra-County-Cases")

In [6]:
soup = BeautifulSoup(r.text)

In [8]:
table = soup.find("table")

In [9]:
table

<table>
<thead>
<tr>
<th id="t8db972bb-69c0-43da-98e4-f947c90ead08c1" scope="col">Total Cases as of 9/21/2021 1:00 pm</th>
<th id="t8db972bb-69c0-43da-98e4-f947c90ead08c2" scope="col">220</th>
</tr>
</thead>
<tbody>
<tr class="textContent">
<td data-th="Total Cases as of 9/21/2021 1:00 pm" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c1"><strong>Active Cases</strong></td>
<td data-th="220" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c2"><strong>3</strong></td>
</tr>
<tr class="alt textContent">
<td data-th="Total Cases as of 9/21/2021 1:00 pm" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c1">Recovered Cases</td>
<td data-th="220" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c2">216</td>
</tr>
<tr class="textContent">
<td data-th="Total Cases as of 9/21/2021 1:00 pm" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c1">Located on the <strong><em>East Side</em></strong> of the County</td>
<td data-th="220" headers="t8db972bb-69c0-43da-98e4-f947c90ead08c2">167</td>
</tr>
<tr class="alt 

In [5]:
img = next(s for s in soup.find_all("img") if "Case Map" in s.attrs.get("alt", ""))

StopIteration: 

In [8]:
url = "http://sierracounty.ca.gov/" + img["src"]

In [9]:
tz = pytz.timezone("America/Los_Angeles")

In [10]:
today = datetime.now(tz).date()

In [11]:
with open(data_dir / "sierra" / f"{today}.png", "wb") as handler:
    handler.write(requests.get(url).content)